In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import gc
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import ast

In [3]:
df = pd.read_csv("../data/olddata/dataset_milvus_server.csv")        # #converters={'Name': pd.eval, 'Dist': pd.eval}
df2 = pd.read_csv("../data/olddata/dataset_milvus_server_2.csv")

cc =  pd.read_csv("../data/creditcard.csv")
cc.Time = np.floor(cc['Time'] / (60*60))

In [4]:
#prepare data
#df_full = pd.concat([df, df2])
df_full = pd.read_csv('../data/ivfflat_server.csv')
df_full = df_full.drop(columns = 'Unnamed: 0')
df_full = df_full.reset_index(drop = True)
df_full['ID'] = df_full.index
df_full['time'] = cc['Time']
df_full['class'] = cc['Class']
df_full.head()

NameError: name 'cc' is not defined

In [13]:
df_full.head()

,name_50,dist_50,ID,time,class
0,"[0, 857, 806, 3009, 867, 562, 3897, 333, 3134,...","[0.0, 5.024761199951172, 5.091703414916992, 5....",0,0.0,0
1,"[1, 3906, 3486, 233, 1925, 59, 916, 3709, 209,...","[0.0, 0.09048068523406982, 0.39853206276893616...",1,0.0,0
2,"[2, 637, 3272, 282, 2662, 2941, 1078, 432, 135...","[0.0, 9.276524543762207, 20.352218627929688, 2...",2,0.0,0
3,"[3, 2401, 1208, 2606, 909, 3958, 2600, 681, 92...","[0.0, 10.868375778198242, 11.147619247436523, ...",3,0.0,0
4,"[4, 438, 2311, 451, 2232, 3611, 3788, 3543, 27...","[0.0, 6.942203998565674, 7.0528130531311035, 7...",4,0.0,0


In [15]:
import ast
#make sure python recognizes the lists
df_full['name_50'] = df_full['name_50'].apply(lambda x: ast.literal_eval(x))
df_full['dist_50'] = df_full['dist_50'].apply(lambda x: ast.literal_eval(x))

In [17]:
#unpack lists into an edgelist

from tqdm import tqdm_notebook
edges = []
dists = []
ID = []
times = []
classess = []

for i in tqdm_notebook(range(0,  int(max(df_full['ID'])+1))):
    namelist = df_full.name_50[i]
    distlist = df_full.dist_50[i]

    for n in range(0,50):
        ID.append(df_full.ID[i])
        times.append(df_full.time[i])
        edges.append(namelist[n])
        dists.append(distlist[n])
        classess.append(df_full['class'][i])

df_edges = pd.DataFrame(list(zip(ID, times, edges, dists, classess)),
               columns =['ID', 'time', 'edges', 'dists', 'class'])

C:\Users\joost\AppData\Local\Temp\ipykernel_15892\693561405.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0,  int(max(df_full['ID'])+1))):


  0%|          | 0/284807 [00:00<?, ?it/s]

In [18]:
#save edges dataset
df_full.to_csv('../data/data_nodes_hnsw.csv')
df_edges.to_csv('../data/data_edges_hnsw.csv')

In [2]:
#make edge dataset into a network, then convert it into a pickle for quick loading later
df = pd.read_csv("../data/data_edges_hnsw.csv")
for t in tqdm_notebook(range(0,48)):
  df_temp = df[df['time'] <= t]
  g_temp = nx.from_pandas_edgelist(df_temp, 'ID', 'edges', ['ID', 'time', 'dists'], create_using=nx.DiGraph())
  nx.write_gpickle(g_temp, "../data/pickles2/myGraph" + str(t) + ".gpickle")
  del df_temp
  del g_temp
  gc.collect()

C:\Users\joost\AppData\Local\Temp\ipykernel_17916\937332648.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for t in tqdm_notebook(range(44,48)):


  0%|          | 0/4 [00:00<?, ?it/s]